# CpNa with GPAW

This is an xHARPy example file for demonstrating how a Hirshfeld Atom Refinement with GPAW is set up. Specifically it is a demonstration of skipping atoms, which are disordered on a special position can be skipped from the calculation with skip_symm in the computation_dict. In this case we are fortunate enough, that this is all it takes to lead the refinement to a reasonable result. Otherwise restraints would be needed, which is something, that is not implemented at the moment

In [9]:
from xharpy import (
    cif2data, xd_hkl2pd, lst2constraint_dict, create_construction_instructions,
    refine, write_cif, write_fcf, write_res
)
import os

In [10]:
output_folder = 'xharpy_output'
if not os.path.exists(output_folder):
    os.mkdir(output_folder)
    
filename = 'xharpy'

In [2]:
atom_table, cell, cell_esd, symm_mats_vecs, symm_strings, wavelength  = cif2data('iam.cif', 0)

In [3]:
hkl = xd_hkl2pd('xd.hkl')

In [4]:
constraint_dict = lst2constraint_dict(
    'iam.lst'
)

In [5]:
refinement_dict = {
    'f0j_source': 'gpaw_mpi',
    'reload_step': 1,
    'core': 'constant',
}

In [6]:
construction_instructions, parameters = create_construction_instructions(
    atom_table=atom_table,
    constraint_dict=constraint_dict,
    refinement_dict=refinement_dict
)


In [12]:
computation_dict = {
    'xc': 'SCAN',
    'txt': os.path.join(output_folder, 'gpaw.txt'),
    'mode': 'fd',
    'h': 0.15,
    'gridinterpolation': 4,
    'average_symmequiv': True,
    'convergence':{'density': 1e-7},
    'kpts': {'size': (1, 1, 1), 'gamma': True},
    'symmetry': {'symmorphic': False},
    'nbands': -2,
    'save_file': os.path.join(output_folder, 'gpaw_result.gpw'),
    'skip_symm': {
        2: [2, 3, 6, 7],
        3: [2, 3, 6, 7],
        4: [2, 3, 6, 7]
    }
}
parameters, var_cov_mat, information = refine(
    cell=cell, 
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict
)

Started refinement at  2022-01-05 14:07:29.183359
Preparing
  calculating the core structure factor by spline for Na
  calculating the core structure factor by spline for N
  calculating the core structure factor by spline for H
  calculating the core structure factor by spline for C
  calculating first atomic form factors


/home/niklas/Documents/har/xharpylib/xharpy/core.py:291: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return un_positions.copy(), type_symbols_symm, np.array(inv_indexes), magmoms_symm


  calculated density with energy -511.4028167516583
  building least squares function
  setting up gradients
step 0: Optimizing scaling
  wR2: 0.085704, number of iterations: 2
  minimizing least squares sum
  wR2: 0.048903, number of iterations: 102
step 1: calculating new structure factors
  calculated density with energy -545.5267843634941
  minimizing least squares sum
  wR2: 0.035719, number of iterations: 91
step 2: calculating new structure factors
  calculated density with energy -544.7679859378709
  minimizing least squares sum
  wR2: 0.037286, number of iterations: 86
step 3: calculating new structure factors
  calculated density with energy -545.3539759719841
  minimizing least squares sum
  wR2: 0.036815, number of iterations: 86
step 4: calculating new structure factors
  calculated density with energy -545.2043799784302
  minimizing least squares sum
  wR2: 0.036938, number of iterations: 85
step 5: calculating new structure factors
  calculated density with energy -545.2

In [13]:
write_fcf(
    fcf_path=os.path.join(output_folder, filename + '.fcf'),
    fcf_dataset='xharpy',
    fcf_mode=4,
    cell=cell,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    symm_strings=symm_strings,
    information=information,
);

In [14]:
write_fcf(
    fcf_path=os.path.join(output_folder, filename + '_6.fcf'),
    fcf_dataset='xharpy_6',
    fcf_mode=6,
    cell=cell,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength,
    refinement_dict=refinement_dict,
    symm_strings=symm_strings,
    information=information,
);

In [15]:
write_res(
    out_res_path=os.path.join(output_folder, filename + '_6.res'),
    in_res_path='iam.lst',
    cell=cell,
    cell_esd=cell_esd,
    construction_instructions=construction_instructions,
    parameters=parameters,
    wavelength=wavelength
)

In [16]:
write_cif(
    output_cif_path=os.path.join(output_folder, filename + '.cif'),
    cif_dataset=filename,
    shelx_cif_path='iam.cif',
    shelx_dataset=0,
    cell=cell,
    cell_esd=cell_esd,
    symm_mats_vecs=symm_mats_vecs,
    hkl=hkl,
    construction_instructions=construction_instructions,
    parameters=parameters,
    var_cov_mat=var_cov_mat,
    refinement_dict=refinement_dict,
    computation_dict=computation_dict,
    information=information
)